In [ ]:
## 初始化程式, 自己設 PREFIX 的路徑
import os
WORK_DIR = "/home/sean/yolov3_tensorflow"
%cd {WORK_DIR}

TRAIN_WEIGHT_PATH = ""

VIDEO_NAME_PATH = "CAM1-24"

!conda activate yolov3_tensorflow
!conda env list

In [ ]:
%cd {WORK_DIR}
!git clone https://github.com/hunglc007/tensorflow-yolov4-tflite.git tf
!cp {TRAIN_WEIGHT} {WORK_DIR}/tf/data/yolo-obj_last.weights

In [ ]:
# 為了 ERROR 而改
!sed -i "s/c1, c2 = (coor\[1\], coor\[0\]), (coor\[3\], coor\[2\])/c1, c2 = (int(coor\[1\]), int(coor\[0\])), (int(coor\[3\]), int(coor\[2\]))/g" {WORK_DIR}/tf/core/utils.py
!sed -i "s/cv2.rectangle(image, c1, (np.float32(c3\[0\]), np.float32(c3\[1\])), bbox_color, -1) #filled/cv2.rectangle(image, c1, (int(np.float32(c3\[0\])), int(np.float32(c3\[1\]))), bbox_color, -1)/" {WORK_DIR}/tf/core/utils.py
!sed -i "s/cv2.putText(image, bbox_mess, (c1\[0\], np.float32(c1\[1\] - 2)), cv2.FONT_HERSHEY_SIMPLEX,/cv2.putText(image, bbox_mess, (c1\[0\], int(np.float32(c1\[1\] - 2))), cv2.FONT_HERSHEY_SIMPLEX,/" {WORK_DIR}/tf/core/utils.py
!sed -i "s/fontScale, (0, 0, 0), bbox_thick \/\/ 2, lineType=cv2.LINE_AA)/fontScale, (0, 0, 0), bbox_thick \/\/ 2, lineType=cv2.LINE_AA)/" {WORK_DIR}/tf/core/utils.py

In [ ]:
!printf "person" > {WORK_DIR}/tf/data/class.names
!sed -i "s/.\/data\/classes\/coco.names/.\/data\/class.names/" {WORK_DIR}/tf/core/config.py

In [ ]:
%cd {WORK_DIR}/tf
# Convert darknet weights to tensorflow
!python save_model.py --weights ./data/yolo-obj_last.weights --output ./checkpoints/yolov4-416 --input_size 416 --model yolov3 --tiny true

In [ ]:
# 為了 ERROR 而改
# https://github.com/hunglc007/tensorflow-yolov4-tflite/pull/346/files/0e5d29e66c263318f166919eea2041630118a179
!sed -i "s/saved_model_loaded = tf.saved_model.load(FLAGS.weights, tags=\[tag_constants.SERVING\])/# saved_model_loaded = tf.saved_model.load(FLAGS.weights, tags=\[tag_constants.SERVING\])/" detectvideo.py
!sed -i "s/infer = saved_model_loaded.signatures\['serving_default'\]/# infer = saved_model_loaded.signatures\['serving_default'\]\n        infer = tf.keras.models.load_model(FLAGS.weights)\n/" detectvideo.py
!sed -i "s/for key, value in pred_bbox.items():/# for key, value in pred_bbox.items():\n            boxes = pred_bbox\[:, :, 0:4\]\n            pred_conf = pred_bbox\[:, :, 4:\]/" detectvideo.py
!sed -i "s/pred_conf = value\[:, :, 4:\]/# pred_conf = value\[:, :, 4:\]/" detectvideo.py
!sed -i "s/boxes = value\[:, :, 0:4\]/# boxes = value\[:, :, 0:4\]/" detectvideo.py

In [ ]:
# 為了不讓圖片顯示出來影響效能, 這邊註解
!sed -i "s/image.show()/# image.show()/" ./detect.py

In [ ]:
%cd {WORK_DIR}/tf
!sed -i "s/frame_id = 0/frame_id = 0\n    print(vid.get(cv2.CAP_PROP_FRAME_COUNT))/" detectvideo.py
!sed -i "s/print(info)/print(info + " " + frame_id)/" detectvideo.py

In [ ]:
%cd {WORK_DIR}/tf
!python detectvideo.py --weights ./checkpoints/yolov4-416 --size 416 --model yolov3 --tiny true --video {VIDEO_NAME_PATH} --dis_cv2_window True --output ./{VIDEO_NAME_PATH}-tinyv3.mp4

In [ ]:
!rm -rf video.zip
!zip video.zip {VIDEO_NAME_PATH}